Objective: Scrape data tables from capstone courier and write into an excle file (one table per sheet) for ease of analysis

*Input Round Below, Then Run All !!!*


In [1]:
round = "6"

### Imports and Global Variables

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import openpyxl

In [3]:
html_path = "C:\\Users\\rzapp\\OneDrive - University of Cincinnati\\Capsim Ferris-40 Documents\\Capstone Couriers\\Courier_Round_" + round + ".htm"
ending_file_path = 'C:\\Users\\rzapp\OneDrive - University of Cincinnati\\Capsim Ferris-40 Documents\\Capstone Couriers\\Courier_Data_Round_' + round + '.xlsx'
list_of_dfs = []

### Read and Parse HTML File

In [4]:
# Read html file contents and transfer into variable
with open(html_path,"r") as file:
    html_contents = file.read()

In [5]:
# Parse html
soup = BeautifulSoup(html_contents,'html.parser')

### Translate HTML Data Into Tables

##### Financial Statements, Production Information, and Segment Tables

In [6]:
# Specific function to convert main tables into data frame
def html_To_DF_Main_Tables(table_results):
    
    try:
        for i in enumerate(table_results):
            if i[0] >= 3:
                df = pd.read_html(str(i[1]))[0]
                df.columns = df.iloc[0]
                df = df[1:]
                list_of_dfs.append(df.fillna(""))
            else:    
                df = pd.read_html(str(i[1]))[0]
                list_of_dfs.append(df.fillna(""))

    except Exception:
        print(Exception)    
    

In [7]:
# Pull financial statements, production  information, and segment product information    
table_results = soup.find_all('table', attrs={'bordercolordark':'#000000', 'width':'655','cellspacing':'0','cellpadding':'1','border':'0'})
html_To_DF_Main_Tables(table_results = table_results)

##### Market Share Report, Financial Statistics, Stock Market, HR, TQM

In [8]:
def Add_Df_To_List(dataFrame):
    
    try:
        dataFrame.columns = dataFrame.fillna("").iloc[0]
        dataFrame = dataFrame[1:].fillna("")
        list_of_dfs.append(dataFrame)

    except Exception:
        print("DataFrame not added to list, error:",Exception)
        

In [9]:
# Convert Actual Market Share (AMS) & Potential Market Share to Dataframe, then add to list_of_dfs
try: 
    AMS_df = pd.read_html(str(soup.find_all('table', attrs={'bordercolordark':'#000000', 'width':'305','cellspacing':'0','cellpadding':'0','border':'0'})[1]))[0]
    PMS_df = pd.read_html(str(soup.find_all('table', attrs={'bordercolordark':'#000000', 'width':'325','cellspacing':'0','cellpadding':'0','border':'0'})[1]))[0]

    Add_Df_To_List(AMS_df)
    Add_Df_To_List(PMS_df)

except Exception:
    print(Exception)

In [10]:
# Financial Statistics and Stock Market Summary
try:    
    fin_stats_df = pd.read_html(str(soup.find_all('table', attrs={'width':'655','cellspacing':'0','cellpadding':'1','border':'0'})[0]))[0]
    stock_summary = pd.read_html(str(soup.find_all('table', attrs={'width':'655','cellspacing':'0','cellpadding':'1','border':'0'})[1]))[0]

    Add_Df_To_List(fin_stats_df)
    list_of_dfs.append(stock_summary)   # No manipulation needed

except Exception:
    print(Exception)

In [11]:
# HR and TQM
try: 
    HR_df = pd.read_html(str(soup.find_all('table', attrs={'bordercolordark':'#000000', 'width':'655','cellspacing':'0','cellpadding':'2','border':'0'})[0]))[0]
    TQM_df = pd.read_html(str(soup.find_all('table', attrs={'bordercolordark':'#000000', 'width':'655','cellspacing':'0','cellpadding':'0','border':'0'})[0]))[0]

    Add_Df_To_List(HR_df)
    Add_Df_To_List(TQM_df)

except Exception:
    print(Exception)

##### Customer Buying Criteria

In [12]:
# Function that gathers buying criteria

def Gather_Buying_Criteria():
    
    try:
        segments = ['Traditional','Low End','High End','Performance','Size']
        attr_list = []
        expect_list = []
        importn_list = []
        segment_list = []

        for i in enumerate(segments):

            # Create a df in a loop because I'm lazy, even though it takes up a ton of power
            df = pd.read_html(str(soup.find_all('table', attrs={'bordercolordark':'#000000', 'width':'325','cellspacing':'0','cellpadding':'1','border':'0'})[i[0]]))[0]
            df.columns = df.fillna("").iloc[0]
            df = df.iloc[1: , 1:].rename(columns = {"":"Attribute"})
            df['Segment'] = i[1]

            # Apply df cols to lists, then append those lists
            attr_list += list(df['Attribute'])
            expect_list += list(df['Expectations'])
            importn_list += list(df['Importance'])
            segment_list += list(df['Segment'])

        zipped_lists = zip(attr_list, expect_list, importn_list, segment_list)
        df = pd.DataFrame(zipped_lists, columns=['Attribute', 'Expectations', 'Importance','Segment'])
        df['Round'] = round
        
        return(df)
    
    except Exception:
        print(Exception)
    

In [13]:
try: 
    list_of_dfs.append(Gather_Buying_Criteria())
    print("successful append")
except Exception:
    print(Exception)

successful append


### Writing DataFrames to an Excel File on SharePoint

In [14]:
table_names = ['Cash Flow Statemment Survey','Balance Sheet Survey','Income Statement Survey','Production Information','Traditional',
    'Low End','High End','Performance','Size','Actual Market Share','Potential Market Share', 'Financial Statistics','Stock Market Summary','HR','TQM','Customer Buying Criteria']

In [15]:
with pd.ExcelWriter(ending_file_path) as writer:
        for i in enumerate(list_of_dfs):
                i[1].to_excel(writer, sheet_name = table_names[i[0]])

Testing New Ideas:
- Autoformat columns before excel
- Set to autorun every (after decisions are officially plugged into system)